# L01: 競馬予想モデル学習（データリーク防止版）

## 修正ポイント
- 時系列を厳密に守る: 各レース時点での「過去のみ」のデータで特徴量計算
- 前走情報をメインに使用（shift で取得済み）

**実行環境**: ローカル（M4 MacBook Air）

---
## 1. 環境設定

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

RACES_PARQUET = PROJECT_ROOT / 'data' / 'processed' / 'races.parquet'
RESULTS_PARQUET = PROJECT_ROOT / 'data' / 'processed' / 'results.parquet'
MODEL_DIR = PROJECT_ROOT / 'models'
MODEL_DIR.mkdir(exist_ok=True)

print(f"データパス: {RESULTS_PARQUET}")

In [ ]:
results_df = pd.read_parquet(RESULTS_PARQUET)
print(f"出走馬データ数: {len(results_df):,}")

---
## 2. 前処理・前走情報追加

**ポイント**: `shift()` を使うと「その馬の過去のレース」情報を安全に取得できる（データリークなし）

In [ ]:
results_df['race_date'] = pd.to_datetime(results_df['race_date'])
results_df['year'] = results_df['race_date'].dt.year

# 完走馬のみ
results_df = results_df[results_df['is_finished'] == True].copy()

# 馬ごとに時系列ソート
results_df = results_df.sort_values(['horse_id', 'race_date']).reset_index(drop=True)

# 前走情報（shift で安全に取得）
print("前走情報を計算中...")
results_df['prev_finish'] = results_df.groupby('horse_id')['finish_position'].shift(1)
results_df['prev_last_3f'] = results_df.groupby('horse_id')['last_3f'].shift(1)
results_df['prev_odds'] = results_df.groupby('horse_id')['odds'].shift(1)
results_df['prev_popularity'] = results_df.groupby('horse_id')['popularity'].shift(1)
results_df['prev_race_date'] = results_df.groupby('horse_id')['race_date'].shift(1)
results_df['days_since_last'] = (results_df['race_date'] - results_df['prev_race_date']).dt.days

# 2走前、3走前
results_df['prev2_finish'] = results_df.groupby('horse_id')['finish_position'].shift(2)
results_df['prev3_finish'] = results_df.groupby('horse_id')['finish_position'].shift(3)
results_df['prev2_last_3f'] = results_df.groupby('horse_id')['last_3f'].shift(2)
results_df['prev3_last_3f'] = results_df.groupby('horse_id')['last_3f'].shift(3)

# 直近3走の平均着順、平均上がり3F
results_df['avg_finish_last3'] = results_df[['prev_finish', 'prev2_finish', 'prev3_finish']].mean(axis=1)
results_df['avg_last3f_last3'] = results_df[['prev_last_3f', 'prev2_last_3f', 'prev3_last_3f']].mean(axis=1)

# 新馬フラグ
results_df['is_debut'] = results_df['prev_finish'].isna().astype(int)

# 累積統計（shift で安全に: expanding を使用）
print("累積統計を計算中（時系列厳守）...")
results_df['horse_cumulative_races'] = results_df.groupby('horse_id').cumcount()  # その時点での通算出走数
results_df['horse_cumulative_wins'] = results_df.groupby('horse_id')['finish_position'].apply(
    lambda x: (x.shift() == 1).cumsum()
).values
results_df['horse_cumulative_place'] = results_df.groupby('horse_id')['finish_position'].apply(
    lambda x: (x.shift() <= 3).cumsum()
).values

# 勝率・複勝率（その時点まで）
results_df['horse_win_rate'] = results_df['horse_cumulative_wins'] / results_df['horse_cumulative_races'].replace(0, np.nan)
results_df['horse_place_rate'] = results_df['horse_cumulative_place'] / results_df['horse_cumulative_races'].replace(0, np.nan)

print("前処理完了")
print(f"データ数: {len(results_df):,}")

In [ ]:
# 騎手・調教師の累積成績も同様に計算
print("騎手・調教師の累積成績を計算中...")

# 日付順にソート
results_df = results_df.sort_values('race_date').reset_index(drop=True)

# 騎手の累積
results_df['jockey_cumulative_races'] = results_df.groupby('jockey_id').cumcount()
results_df['jockey_cumulative_wins'] = results_df.groupby('jockey_id')['finish_position'].apply(
    lambda x: (x.shift() == 1).cumsum()
).values
results_df['jockey_cumulative_place'] = results_df.groupby('jockey_id')['finish_position'].apply(
    lambda x: (x.shift() <= 3).cumsum()
).values
results_df['jockey_win_rate'] = results_df['jockey_cumulative_wins'] / results_df['jockey_cumulative_races'].replace(0, np.nan)
results_df['jockey_place_rate'] = results_df['jockey_cumulative_place'] / results_df['jockey_cumulative_races'].replace(0, np.nan)

# 調教師の累積
results_df['trainer_cumulative_races'] = results_df.groupby('trainer_id').cumcount()
results_df['trainer_cumulative_wins'] = results_df.groupby('trainer_id')['finish_position'].apply(
    lambda x: (x.shift() == 1).cumsum()
).values
results_df['trainer_cumulative_place'] = results_df.groupby('trainer_id')['finish_position'].apply(
    lambda x: (x.shift() <= 3).cumsum()
).values
results_df['trainer_win_rate'] = results_df['trainer_cumulative_wins'] / results_df['trainer_cumulative_races'].replace(0, np.nan)
results_df['trainer_place_rate'] = results_df['trainer_cumulative_place'] / results_df['trainer_cumulative_races'].replace(0, np.nan)

print("完了")

---
## 3. データ分割

- **Train**: 2010〜2024年
- **Test**: 2025年〜

In [ ]:
TRAIN_END_YEAR = 2024

train_df = results_df[results_df['year'] <= TRAIN_END_YEAR].copy()
test_df = results_df[results_df['year'] > TRAIN_END_YEAR].copy()

# 追加特徴量
train_df['surface_encoded'] = train_df['surface'].map({'芝': 0, 'ダート': 1}).fillna(-1)
test_df['surface_encoded'] = test_df['surface'].map({'芝': 0, 'ダート': 1}).fillna(-1)
train_df['target'] = (train_df['finish_position'] <= 3).astype(int)
test_df['target'] = (test_df['finish_position'] <= 3).astype(int)

print(f"Train: {len(train_df):,} ({train_df['year'].min()}〜{train_df['year'].max()})")
print(f"Test: {len(test_df):,} ({test_df['year'].min()}〜{test_df['year'].max()})")

---
## 4. モデル学習

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# オッズなしモデル（純粋予測）
FEATURE_COLS_NO_ODDS = [
    # レース情報
    'distance', 'surface_encoded', 'level_score',
    'horse_number', 'gate_number', 'impost',
    # 前走情報（shift で取得 = リークなし）
    'prev_finish', 'prev_last_3f', 'days_since_last', 'is_debut',
    'prev2_finish', 'prev3_finish',
    'avg_finish_last3', 'avg_last3f_last3',
    # 累積統計（cumsum + shift = リークなし）
    'horse_cumulative_races', 'horse_win_rate', 'horse_place_rate',
    'jockey_cumulative_races', 'jockey_win_rate', 'jockey_place_rate',
    'trainer_cumulative_races', 'trainer_win_rate', 'trainer_place_rate',
]

# オッズありモデル
FEATURE_COLS_WITH_ODDS = FEATURE_COLS_NO_ODDS + ['popularity', 'odds']

print("オッズなしモデル特徴量:", len(FEATURE_COLS_NO_ODDS), "個")
print("オッズありモデル特徴量:", len(FEATURE_COLS_WITH_ODDS), "個")

In [ ]:
# NaN埋め
for col in FEATURE_COLS_WITH_ODDS:
    if col in train_df.columns:
        train_df[col] = train_df[col].fillna(0)
        test_df[col] = test_df[col].fillna(0)

train_clean = train_df.dropna(subset=['target'])
test_clean = test_df.dropna(subset=['target'])

print(f"Train (clean): {len(train_clean):,}")
print(f"Test (clean): {len(test_clean):,}")

In [ ]:
def train_and_evaluate(train_data, test_data, feature_cols, model_name):
    print(f"\n{'='*50}")
    print(f"🎯 {model_name}")
    print(f"{'='*50}")
    
    X_train = train_data[feature_cols]
    y_train = train_data['target']
    X_test = test_data[feature_cols]
    y_test = test_data['target']
    
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 15,  # 過学習防止のため減らす
        'min_data_in_leaf': 100,  # 過学習防止
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'verbose': -1,
        'random_state': 42,
    }
    
    train_ds = lgb.Dataset(X_tr, label=y_tr)
    val_ds = lgb.Dataset(X_val, label=y_val, reference=train_ds)
    
    model = lgb.train(
        params,
        train_ds,
        num_boost_round=1000,
        valid_sets=[train_ds, val_ds],
        valid_names=['train', 'valid'],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )
    
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    print(f"\n📊 Test AUC: {auc:.4f}")
    
    test_with_pred = test_data.copy()
    test_with_pred['pred_proba'] = y_pred
    
    # 的中率
    def calc_hit(group, top_n=3):
        top_preds = group.nlargest(top_n, 'pred_proba')
        return (top_preds['finish_position'] <= 3).any()
    
    race_hit = test_with_pred.groupby('race_id').apply(calc_hit)
    print(f"🎯 上位3頭予測の的中率: {race_hit.mean():.1%}")
    
    # 単勝回収率
    def sim_returns(group):
        top = group.nlargest(1, 'pred_proba').iloc[0]
        return top['odds'] * 100 if top['finish_position'] == 1 else 0
    
    returns = test_with_pred.groupby('race_id').apply(sim_returns)
    return_rate = returns.sum() / (len(returns) * 100) * 100
    print(f"💰 単勝回収率: {return_rate:.1f}%")
    
    # 三連複
    def calc_sanren(group):
        top5 = group.nlargest(5, 'pred_proba')
        actual_top3 = set(group.nsmallest(3, 'finish_position')['horse_id'])
        pred_top5 = set(top5['horse_id'])
        return len(actual_top3 & pred_top5) >= 3
    
    sanren = test_with_pred.groupby('race_id').apply(calc_sanren)
    print(f"🏇 三連複的中率（5頭選択）: {sanren.mean():.1%}")
    
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importance()
    }).sort_values('importance', ascending=False)
    
    return {
        'model': model,
        'auc': auc,
        'hit_rate': race_hit.mean(),
        'return_rate': return_rate,
        'sanren_rate': sanren.mean(),
        'importance': importance,
        'predictions': test_with_pred
    }

In [ ]:
result_no_odds = train_and_evaluate(
    train_clean, test_clean,
    FEATURE_COLS_NO_ODDS,
    "オッズなしモデル（純粋予測）"
)

In [ ]:
result_with_odds = train_and_evaluate(
    train_clean, test_clean,
    FEATURE_COLS_WITH_ODDS,
    "オッズありモデル（速報オッズ込み）"
)

---
## 5. モデル比較

In [ ]:
print("\n" + "="*60)
print("📊 モデル比較サマリー")
print("="*60)

comparison = pd.DataFrame({
    '指標': ['AUC', '上位3頭的中率', '三連複的中率（5頭選択）', '単勝回収率'],
    'オッズなし': [
        f"{result_no_odds['auc']:.4f}",
        f"{result_no_odds['hit_rate']:.1%}",
        f"{result_no_odds['sanren_rate']:.1%}",
        f"{result_no_odds['return_rate']:.1f}%"
    ],
    'オッズあり': [
        f"{result_with_odds['auc']:.4f}",
        f"{result_with_odds['hit_rate']:.1%}",
        f"{result_with_odds['sanren_rate']:.1%}",
        f"{result_with_odds['return_rate']:.1f}%"
    ]
})

print(comparison.to_string(index=False))

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, (name, result) in zip(axes, [
    ('オッズなしモデル', result_no_odds),
    ('オッズありモデル', result_with_odds)
]):
    imp = result['importance'].head(12)
    ax.barh(imp['feature'], imp['importance'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{name}\nTop 12 Features')
    ax.invert_yaxis()

plt.tight_layout()
plt.show()

---
## 6. モデル保存

In [ ]:
import pickle

with open(MODEL_DIR / 'model_no_odds.pkl', 'wb') as f:
    pickle.dump(result_no_odds['model'], f)

with open(MODEL_DIR / 'model_with_odds.pkl', 'wb') as f:
    pickle.dump(result_with_odds['model'], f)

config = {
    'feature_cols_no_odds': FEATURE_COLS_NO_ODDS,
    'feature_cols_with_odds': FEATURE_COLS_WITH_ODDS,
    'train_end_year': TRAIN_END_YEAR,
}

with open(MODEL_DIR / 'model_config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("✅ モデル保存完了")